# public API

### instantiation of exchange

In [1]:
import pandas as pd
import numpy as np
import ccxt

In [2]:
exchange = ccxt.binanceusdm({
	'apiKey': '296015e94a672f83bfb4076908741a255cd5c9f192da047d12eb63cff4c73a7e',
	'secret': '4ecd9157f7b1a8486742374e9627d4ae4cfca3b997d1532ef447dc915fad62f5',
})
# turn on testnet
# hasattr(exchange, 'set_sandbox_mode')
exchange.set_sandbox_mode(True)


In [4]:
exchange.timeframes

{'1s': '1s',
 '1m': '1m',
 '3m': '3m',
 '5m': '5m',
 '15m': '15m',
 '30m': '30m',
 '1h': '1h',
 '2h': '2h',
 '4h': '4h',
 '6h': '6h',
 '8h': '8h',
 '12h': '12h',
 '1d': '1d',
 '3d': '3d',
 '1w': '1w',
 '1M': '1M'}

In [ ]:
exchange.fetch_ticker(symbol='ETHUSDT')

In [5]:
df = pd.DataFrame(exchange.fetch_ohlcv(symbol='ETHUSDT', timeframe='2h', limit= 1000))

In [6]:
df.describe()

,0,1,2,3,4,5
count,1.000000e+03,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,1.675595e+12,1524.960530,1541.62567,1509.454470,1525.537740,25637.946853
std,2.079500e+09,160.611278,166.24499,157.412678,160.543546,16020.788489
min,1.671998e+12,1187.270000,1191.58000,1165.000000,1187.290000,3929.882000
25%,1.673797e+12,1448.977500,1471.66000,1428.190000,1450.262500,15306.446000
50%,1.675595e+12,1574.295000,1590.07500,1560.000000,1574.775000,21465.512500
75%,1.677393e+12,1641.755000,1660.00000,1626.777500,1641.655000,30831.360750
max,1.679191e+12,1792.000000,1817.87000,1770.010000,1792.780000,153886.347000


### get data && clean data

In [10]:
exchange.rateLimit/1000

0.05

In [3]:
import time


def get_history(symbol, interval, start= None, limit = 1000):
	if start:
		start = exchange.parse8601(start)

	data = exchange.fetch_ohlcv(symbol=symbol, timeframe=interval, since=start, limit=limit) 
	lastBarActual = data[-1][0] # timestamp of last loaded bar
	# timestamp of current bar
	lastBarCurrent = exchange.fetch_ohlcv(symbol=symbol, timeframe=interval, limit=1)[-1][0]
	# if lastBarActual != lastBarCurrent => pull the next 1000 bars
	while lastBarActual != lastBarCurrent:
		time.sleep(0.1)
		data += exchange.fetch_ohlcv(symbol=symbol, timeframe=interval, since=lastBarActual, limit=limit)[1:]
		lastBarActual = data[-1][0]
		lastBarCurrent = exchange.fetch_ohlcv(symbol=symbol, timeframe=interval, limit=1)[-1][0]

	df = pd.DataFrame(data)
	df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
	df.Date= pd.to_datetime(df.Date, unit='ms')
	df.set_index('Date', inplace=True)
	
	return df

In [15]:
start = "2022-01-01 00:00:00"
# since start but max 1000 bars
df = get_history('ETHUSDT', '2h', start=start, limit=1000)
df.info

<bound method DataFrame.info of                         Open     High      Low    Close     Volume
Date                                                              
2022-01-01 00:00:00  3677.60  3747.39  3677.26  3725.91  17634.510
2022-01-01 02:00:00  3725.66  3747.37  3356.65  3723.91  12856.258
2022-01-01 04:00:00  3723.92  3763.38  3703.69  3736.80  15003.712
2022-01-01 06:00:00  3736.78  3742.47  3703.99  3716.73   8329.390
2022-01-01 08:00:00  3716.70  3732.58  3710.06  3712.75   4715.746
...                      ...      ...      ...      ...        ...
2023-03-18 20:00:00  1713.59  1729.00  1682.00  1682.00   7056.861
2023-03-18 22:00:00  1689.31  1729.00  1676.00  1705.46  11071.876
2023-03-19 00:00:00  1705.46  1785.00  1700.00  1777.53  46849.343
2023-03-19 02:00:00  1777.72  1785.00  1725.00  1771.59  14155.651
2023-03-19 04:00:00  1763.19  1776.38  1762.13  1762.13    955.794

[5307 rows x 5 columns]>

In [18]:
df.index.is_unique

True

## streaming data

In [20]:
# basic idea to streaming klines with no requirement for low latency
for i in range(10):
    data = exchange.fetch_ohlcv(symbol='ETHUSDT', timeframe='2h', limit= 1)
    print(data)
    time.sleep(1)

[[1679198400000, 1763.19, 1777.46, 1761.08, 1776.72, 1478.979]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1479.73]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1480.045]]
[[1679198400000, 1763.19, 1777.46, 1761.08, 1777.46, 1480.045]]


In [20]:
def start_klines(callback, symbol, interval):
	global running # global variable to control the loop externally
	running = True
 
	while running == True:
		res = exchange.fetch_ohlcv(symbol=symbol, timeframe=interval, limit= 1)
		if len(res) == 0:
			print('no data')
		else:
			callback(res)
		time.sleep(1)

In [13]:
df = pd.DataFrame(columns= ["Open", "High", "Low", "Close", "Volume"])
df

,Open,High,Low,Close,Volume


In [24]:
def stream_candles(res):
    # define how to process the data

    # extract data from res
    start_time = pd.to_datetime(res[0][0], unit='ms') # res:[[1679198400000, 1763.19, 1777.46, 1761.08, 1776.72, 1478.979]]
    first = res[0][1]
    high = res[0][2]
    low = res[0][3]
    close = res[0][4]
    volume = res[0][5]

    # feed the df
    df.loc[start_time] = [first, high, low, close, volume]

    # print sth
    print(".", end = "", flush = True)

In [6]:
def stop_stream():
	global running
	running = False

### multithreading

In [8]:
from threading import Thread

In [25]:
thread = Thread(target=start_klines, args=(stream_candles, 'ETHUSDT', '1m'))
thread

<Thread(Thread-6, initial)>

In [26]:
thread.start()

....................

In [27]:
stop_stream()

In [15]:
thread

<Thread(Thread-4, initial)>

In [16]:
thread.start()

[[1679293020000, 1749.95, 1751.0, 1749.95, 1751.0, 36.759]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1749.95, 36.988]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1749.95, 36.988]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1749.95, 36.988]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1749.95, 36.988]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1749.95, 36.988]]
[[1679293020000, 1749.95, 1751.0, 1749.95, 1751.0, 37.217]]


In [17]:
stop_stream()

[[1679293080000, 1751.0, 1751.0, 1749.95, 1749.95, 57.732]]


In [28]:
df

,Open,High,Low,Close,Volume
2023-03-20 06:19:00,1751.0,1751.0,1749.95,1750.98,95.629
2023-03-20 06:20:00,1751.0,1751.0,1750.98,1750.98,1.775
